# GIFT Hybrid Eigenvalue Computation

Combine les meilleures approches:
1. **PINN avec Rayleigh quotient** - minimise directement R[psi]
2. **Fourier features** - resout le biais spectral
3. **Matrix-free Lanczos** - validation directe

Objectif: tester si lambda_1 * H* = 14 sur metriques G2 non-separables

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import eigsh, LinearOperator
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

## 1. PINN avec Rayleigh Quotient et Fourier Features

In [ ]:
class FourierFeatures(nn.Module):
    """Random Fourier features pour eviter le biais spectral."""
    def __init__(self, input_dim=7, num_freq=32, scale=2.0):
        super().__init__()
        B = torch.randn(num_freq, input_dim) * scale
        self.register_buffer('B', B)
        self.output_dim = 2 * num_freq
    
    def forward(self, x):
        proj = 2 * np.pi * x @ self.B.T
        return torch.cat([torch.cos(proj), torch.sin(proj)], dim=-1)


class EigenfunctionNet(nn.Module):
    """Reseau pour approximer la fonction propre psi."""
    def __init__(self, num_freq=32, hidden_dim=128, n_layers=4):
        super().__init__()
        self.fourier = FourierFeatures(7, num_freq, scale=2.0)
        
        layers = []
        in_dim = self.fourier.output_dim
        for _ in range(n_layers):
            layers.extend([
                nn.Linear(in_dim, hidden_dim),
                nn.SiLU(),
                nn.LayerNorm(hidden_dim)
            ])
            in_dim = hidden_dim
        layers.append(nn.Linear(hidden_dim, 1))
        self.net = nn.Sequential(*layers)
    
    def forward(self, x):
        h = self.fourier(x)
        return self.net(h).squeeze(-1)

In [ ]:
def g2_metric_diagonal(x, H_star):
    """
    Metrique G2 simplifiee mais non-triviale.
    
    g_ii(x) = 1 + alpha * cos(sum_j x_j) + beta * prod_j sin(x_j)
    
    Cela couple toutes les dimensions (non-separable).
    """
    alpha = 0.3  # Couplage global
    beta = 0.1   # Couplage non-lineaire
    
    # Terme de couplage global
    sum_x = x.sum(dim=-1, keepdim=True)  # (N, 1)
    cos_term = alpha * torch.cos(sum_x)  # (N, 1)
    
    # Terme non-lineaire (produit de sinus)
    sin_prod = torch.sin(x).prod(dim=-1, keepdim=True)  # (N, 1)
    nonlin_term = beta * sin_prod  # (N, 1)
    
    # Metrique diagonale mais couplee spatialement
    g_diag = 1.0 + cos_term + nonlin_term  # (N, 1)
    g_diag = g_diag.expand(-1, 7)  # (N, 7)
    
    # Normalisation pour Vol ~ 1
    # scale = (H_star / 14.0) ** (1/7)  # Optional scaling
    
    return g_diag


def compute_rayleigh_quotient(psi_net, x, H_star):
    """
    Calcule R[psi] = integral(|grad psi|^2_g dV) / integral(psi^2 dV)
    
    C'est directement l'estimation de lambda_1.
    """
    x = x.requires_grad_(True)
    psi = psi_net(x)
    
    # Centrer (mode constant = 0)
    psi_centered = psi - psi.mean()
    
    # Gradient via autograd
    grad_psi = torch.autograd.grad(
        psi_centered.sum(), x, create_graph=True, retain_graph=True
    )[0]  # (N, 7)
    
    # Metrique
    g_diag = g2_metric_diagonal(x, H_star)  # (N, 7)
    g_inv = 1.0 / g_diag  # Pour metrique diagonale
    sqrt_det_g = torch.sqrt(g_diag.prod(dim=-1))  # (N,)
    
    # Norme du gradient avec metrique: |grad psi|^2_g = g^{ij} d_i psi d_j psi
    grad_norm_sq = (g_inv * grad_psi ** 2).sum(dim=-1)  # (N,)
    
    # Integrales Monte Carlo
    numerator = (grad_norm_sq * sqrt_det_g).mean()
    denominator = (psi_centered ** 2 * sqrt_det_g).mean()
    
    R = numerator / (denominator + 1e-10)
    return R, psi_centered

In [ ]:
def train_pinn_rayleigh(H_star, n_epochs=2000, batch_size=512, lr=1e-3):
    """Entrainer le PINN avec perte Rayleigh quotient."""
    model = EigenfunctionNet(num_freq=32, hidden_dim=128, n_layers=4).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs, eta_min=1e-5)
    
    history = []
    
    for epoch in range(n_epochs):
        # Echantillonner sur le tore [0, 2pi]^7
        x = torch.rand(batch_size, 7, device=device) * 2 * np.pi
        
        R, _ = compute_rayleigh_quotient(model, x, H_star)
        
        optimizer.zero_grad()
        R.backward()
        optimizer.step()
        scheduler.step()
        
        history.append(R.item())
        
        if (epoch + 1) % 500 == 0:
            print(f'Epoch {epoch+1}: R = {R.item():.6f}, lambda*H* = {R.item()*H_star:.2f}')
    
    return model, history

## 2. Test sur differentes valeurs de H*

In [ ]:
print('='*60)
print('PINN avec Rayleigh Quotient - Metrique G2 couplee')
print('='*60)

results_pinn = []

for name, b2, b3 in [('K7', 21, 77), ('J1', 12, 43), ('Kov', 0, 71)]:
    H_star = b2 + b3 + 1
    print(f'\n--- {name} (H*={H_star}) ---')
    
    model, history = train_pinn_rayleigh(H_star, n_epochs=2000)
    
    # Valeur finale
    lambda_1 = history[-1]
    product = lambda_1 * H_star
    
    print(f'Final: lambda_1 = {lambda_1:.6f}, lambda_1 * H* = {product:.2f}')
    results_pinn.append((name, H_star, lambda_1, product))

In [ ]:
# Resume PINN
print('\n' + '='*60)
print('RESUME PINN (Rayleigh + Fourier + Metrique couplee)')
print('='*60)
print(f"{'Manifold':<10} {'H*':>5} {'lambda_1':>12} {'lambda*H*':>12}")
print('-'*45)
products = []
for name, H_star, lam, prod in results_pinn:
    print(f'{name:<10} {H_star:>5} {lam:>12.6f} {prod:>12.2f}')
    products.append(prod)
print('-'*45)
print(f"{'Mean':<10} {'':<5} {'':<12} {np.mean(products):>12.2f}")
print(f"{'Std':<10} {'':<5} {'':<12} {np.std(products):>12.2f}")
print(f'\nCible GIFT: 14.00')

## 3. Validation avec methode directe (Lanczos incrementale)

In [ ]:
def build_laplacian_coupled(n, H_star):
    """
    Construire le Laplacien 7D avec metrique couplee.
    Utilise LinearOperator pour eviter de stocker la matrice.
    
    Pour n petit (n=3-5), c'est faisable.
    """
    dim = 7
    N = n ** dim
    h = 2 * np.pi / n
    
    # Precompute coordinates
    coords_1d = np.linspace(0, 2*np.pi, n, endpoint=False)
    grids = np.meshgrid(*[coords_1d]*dim, indexing='ij')
    coords = np.stack([g.ravel() for g in grids], axis=1)  # (N, 7)
    
    # Metrique (version numpy)
    alpha, beta = 0.3, 0.1
    sum_x = coords.sum(axis=1, keepdims=True)
    sin_prod = np.sin(coords).prod(axis=1, keepdims=True)
    g_diag = 1.0 + alpha * np.cos(sum_x) + beta * sin_prod
    g_diag = np.tile(g_diag, (1, dim))  # (N, 7)
    sqrt_det_g = np.sqrt(g_diag.prod(axis=1))  # (N,)
    
    def matvec(v):
        """Appliquer le Laplacien courbe sans stocker la matrice."""
        u = v.reshape([n]*dim)
        result = np.zeros_like(u)
        
        for d in range(dim):
            # Derivees secondes dans direction d (periodiques)
            u_plus = np.roll(u, -1, axis=d)
            u_minus = np.roll(u, 1, axis=d)
            
            # Laplacien standard
            laplacian_d = (u_plus - 2*u + u_minus) / h**2
            
            # Ponderer par metrique inverse
            g_d = g_diag[:, d].reshape([n]*dim)
            result += laplacian_d / g_d
        
        # Diviser par sqrt(det g)
        result = result.ravel() / sqrt_det_g
        
        return -result  # Convention: -Delta (positif)
    
    return LinearOperator((N, N), matvec=matvec, dtype=np.float64)


def direct_eigenvalue(n, H_star):
    """Calculer lambda_1 par methode directe."""
    L = build_laplacian_coupled(n, H_star)
    
    # eigsh pour valeurs propres les plus petites
    evals, evecs = eigsh(L, k=3, which='SM', tol=1e-6)
    evals = np.sort(evals)
    
    # lambda_1 = premiere valeur propre non-nulle
    lambda_1 = evals[evals > 1e-6][0] if (evals > 1e-6).any() else evals[1]
    return lambda_1

In [ ]:
print('='*60)
print('Methode directe (Lanczos matrix-free)')
print('='*60)

results_direct = []

for n in [3, 4, 5]:  # Grilles de plus en plus fines
    N_total = n**7
    print(f'\n--- Grille n={n} ({N_total:,} points) ---')
    
    for name, b2, b3 in [('K7', 21, 77)]:
        H_star = b2 + b3 + 1
        
        try:
            lambda_1 = direct_eigenvalue(n, H_star)
            product = lambda_1 * H_star
            print(f'{name} (H*={H_star}): lambda_1 = {lambda_1:.6f}, lambda*H* = {product:.2f}')
            results_direct.append((n, name, H_star, lambda_1, product))
        except Exception as e:
            print(f'{name}: Erreur - {e}')

In [ ]:
# Richardson extrapolation si on a plusieurs grilles
if len(results_direct) >= 2:
    print('\n' + '='*60)
    print('Richardson Extrapolation')
    print('='*60)
    
    # Extrapoler vers h -> 0
    ns = [r[0] for r in results_direct]
    prods = [r[4] for r in results_direct]
    
    if len(ns) >= 2:
        # Ordre 2: lambda(h) = lambda_exact + c*h^2
        # Richardson: lambda_exact ~ (n2^2 * l2 - n1^2 * l1) / (n2^2 - n1^2)
        n1, n2 = ns[-2], ns[-1]
        l1, l2 = prods[-2], prods[-1]
        
        lambda_extrap = (n2**2 * l2 - n1**2 * l1) / (n2**2 - n1**2)
        print(f'Extrapolation (n={n1}, n={n2}): lambda*H* -> {lambda_extrap:.2f}')

## 4. Analyse et Conclusion

In [ ]:
print('='*60)
print('ANALYSE FINALE')
print('='*60)

print('''
Le modele TCS separable donne lambda*H* ~ 19.6
La metrique G2 couplee (cos + sin prod) change-t-elle ce resultat?

Observations cles:
1. Le Rayleigh quotient PINN converge vers une valeur stable
2. La methode directe confirme (ou infirme) cette valeur
3. Le couplage spatial modifie-t-il lambda*H* vers 14?

Interpretation:
- Si lambda*H* ~ 14: La formule GIFT emerge de la geometrie couplee
- Si lambda*H* != 14: Le modele simplifie ne capture pas la vraie G2
''')

if results_pinn:
    mean_pinn = np.mean([r[3] for r in results_pinn])
    print(f'PINN Rayleigh: mean(lambda*H*) = {mean_pinn:.2f}')
    print(f'GIFT target: 14.00')
    print(f'Ecart: {abs(mean_pinn - 14)/14*100:.1f}%')